# This project is about marketing.

We will work in a dataset of GooglePlay from Kaggle.

In this dataset we are going to work with 13 columns and 10841 lines.
We want to know which is the better and the worst application according to ranting and see what is the diffence between free and paid in installed applications.

...

In [59]:
import numpy as np
import matplotlib.pyplot
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier

In [60]:
df2 = pd.read_csv('C:/Users/Vitorioso1/Documents/Project-A/googleplaystore.csv/googleplaystore.csv')
df2.head(2)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up


In [61]:
df2.isna().sum()

App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
Current Ver          8
Android Ver          3
dtype: int64

In [62]:
d33 = df2.dropna()
d33.shape

(9360, 13)

In [63]:
d33.tail(5)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10834,FR Calculator,FAMILY,4.0,7,2.6M,500+,Free,0,Everyone,Education,"June 18, 2017",1.0.0,4.1 and up
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,Education,"July 6, 2018",1.0,4.1 and up
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device
10840,iHoroscope - 2018 Daily Horoscope & Astrology,LIFESTYLE,4.5,398307,19M,"10,000,000+",Free,0,Everyone,Lifestyle,"July 25, 2018",Varies with device,Varies with device


In [64]:
d33.describe()

,Rating
count,9360.000000
mean,4.191838
std,0.515263
min,1.000000
25%,4.000000
50%,4.300000
75%,4.500000
max,5.000000


##### We found out more than 1400 missing values and we have it deleted in the column Rating. 

Our data will have two models predictors only for ensure, and later we will make tests of accuracy.

In [65]:
d33.dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

##### In the column Reviews, we have the object called '3.0M' in the index 10472 and I don't know what this means so we will fill it with np.NAN, we need to do this to transform the column to a numeric type.

In [66]:
df = d33
pd.to_numeric(df['Reviews'])
df.head(2)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up


In [67]:
len(df.groupby('Size').sum())

413

In [68]:
len(df.groupby('Type').sum())

2

In [69]:
len(df.groupby('Reviews').sum())

5990

In [70]:
df['Reviews'] = df['Reviews'].astype('float')

<ipython-input-70-efea75fdc1d8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Reviews'] = df['Reviews'].astype('float')


In [71]:
len(df.groupby('Category').sum())


33

In [72]:
len(df.groupby('Genres'))

115

In [73]:
len(df.groupby('Content Rating').sum())

6

##### In the lines above, we can see the numbers of different types that we have in some columns, as we can see for example the column Category has 33 different types of object. 

In [74]:
df.head(2)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159.0,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967.0,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up


In [75]:
df.dtypes

App                object
Category           object
Rating            float64
Reviews           float64
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

##### We need to do some changes in our dataset for make a effective model.

In [76]:
df4 = df
df4['Installs'] = df4['Installs'].str.replace(r'\D', '')

<ipython-input-76-a22daec4c2bc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['Installs'] = df4['Installs'].str.replace(r'\D', '')


In [77]:
df4['Installs'] = df4['Installs'].astype('int')

<ipython-input-77-1887f8d35f62>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['Installs'] = df4['Installs'].astype('int')


In [78]:
df4.dtypes

App                object
Category           object
Rating            float64
Reviews           float64
Size               object
Installs            int32
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

##### Now we will create the datasets that will be trained,  and later we will need to encoder the columns object to train and predict it, let's use the model LabelEncoder to do it.

In [160]:
y = df4['Rating'].astype('int')
y.dtypes

dtype('int32')

In [161]:
X = df4[['Reviews', 'Type', 'Content Rating']].values

In [162]:
df4.groupby('Type')['Rating'].sum()

Type
Free    36483.5
Paid     2752.1
Name: Rating, dtype: float64

In [163]:
df4.groupby('Content Rating')['Rating'].sum()

Content Rating
Adults only 18+       12.9
Everyone           31038.5
Everyone 10+        1690.1
Mature 17+          1900.9
Teen                4589.1
Unrated                4.1
Name: Rating, dtype: float64

In [164]:
X[:,2]

array(['Everyone', 'Everyone', 'Everyone', ..., 'Everyone', 'Mature 17+',
       'Everyone'], dtype=object)

In [165]:
lbc = preprocessing.LabelEncoder()

In [166]:
lbc.fit(['Free', 'Paid'])
X[:,1] = lbc.transform(X[:,1])

In [167]:
lbct = preprocessing.LabelEncoder()
lbct.fit(['Adults only 18+','Everyone', 'Everyone 10+','Mature 17+','Teen', 'Unrated'])
X[:,2] = lbct.transform(X[:,2])

In [168]:
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size = 0.35, random_state = 4)

##### We will create our model decision tree and test the accuracy of it.

In [211]:
dtc = DecisionTreeClassifier(criterion = 'entropy', max_depth = 6)

In [212]:
scoresD = cross_val_score(dtc, X, y, scoring = 'accuracy')
print('The accuracy of our model in this data is:{0:.1f}%'.format(np.mean(scoresD)*100))

The accuracy of our model in this data is:76.0%


As we can see the accuracy of the decision tree in this data is 76%.

##### Let's create the model support vitual machine and the model random forest and test them.

In [213]:
ssv = svm.SVC(kernel = 'rbf')

In [214]:
scoresS = cross_val_score(ssv, X, y, scoring = 'accuracy')
print('The accuracy of svm in this data is:{0:.1f}%'.format(np.mean(scoresS)*100))

The accuracy of svm in this data is:75.7%


In [215]:
rfc = RandomForestClassifier(n_estimators = 300, criterion = 'entropy')

In [216]:
scoresR = cross_val_score(rfc, X, y, scoring = 'accuracy')
print('The accuracy of random forest in this data is:{0:.1f}%'.format(np.mean(scoresR)*100))

The accuracy of random forest in this data is:68.1%


##### In the lines above, we can see that the svm have 75.7% and random forest have the lowest percentage of accuracy it is 68.1%.

##### Let's use the model with better percentage, that is DecisionTree.

In [217]:
dtc.fit(X_trainset, y_trainset)

DecisionTreeClassifier(criterion='entropy', max_depth=6)

In [218]:
y_pred = dtc.predict(X_testset)

In [219]:
y_pred[:3]

array([4, 4, 4])

In [220]:
print('The minimum rate of the predicted data is:', y_pred.min())
print('The maximum rate of the predicted data is:', y_pred.max())
print('The average rate of the predicted data is:', y_pred.mean())
print('The standard deviation of the predicted data is:', y_pred.std(ddof = 1))

The minimum rate of the predicted data is: 2
The maximum rate of the predicted data is: 5
The average rate of the predicted data is: 4.018315018315018
The standard deviation of the predicted data is: 0.19054049190072608


In [221]:
 print('The accuracy of the predicted data is:{0:.1f}%'.format(metrics.accuracy_score(y_testset, y_pred)*100))

The accuracy of the predicted data is:76.7%


##### Now we have a model that estimate the rate of application based in reviews, type that is if the app is free or paid and Content Rating with more than 76% of accuracy.

...

##### With this model a company of applications can estimate which type of application have the greater chance to have many downloads. A company of application could use this model to describe whether create a application paid is better or worst, or the company could use this model to describe whether create application with the "content rating of everyone 10+" is advantageous for exemple.

# Thank you for work with me.